In [19]:
import os

In [13]:
walk = [file[0] for file in os.walk('/home/pierre/.ros/log')]

In [14]:
walk[1:]

['/home/pierre/.ros/log/2021-11-30-14-40-59-661651-reachy-xprize-29832',
 '/home/pierre/.ros/log/2021-11-30-10-33-58-737989-reachy-xprize-21188',
 '/home/pierre/.ros/log/2021-11-30-10-42-51-138714-reachy-xprize-24653',
 '/home/pierre/.ros/log/2021-11-30-14-55-22-925747-reachy-xprize-31046',
 '/home/pierre/.ros/log/2021-11-30-10-35-19-708466-reachy-xprize-24385',
 '/home/pierre/.ros/log/2021-11-30-14-41-51-101839-reachy-xprize-30005',
 '/home/pierre/.ros/log/2021-11-30-14-42-31-368355-reachy-xprize-30233']

In [59]:
with open(walk[-1] + '/launch.log') as log_file:
    lines = log_file.readlines()

In [18]:
error_messages = ['Errno', 'ERROR']

In [19]:
for line in lines:
    for error_msg in error_messages:
        if error_msg in line:
            print(line)

1638279751.8536079 [camera_zoom_service-7] FileNotFoundError: [Errno 2] No such file or directory: '/dev/ttyACM0'

1638279751.8550231 [camera_zoom_service-7] serial.serialutil.SerialException: [Errno 2] could not open port /dev/ttyACM0: [Errno 2] No such file or directory: '/dev/ttyACM0'

1638279751.9388404 [ERROR] [camera_zoom_service-7]: process has died [pid 30247, exit code 1, cmd '/home/pierre/reachy_ws/install/reachy_controllers/lib/reachy_controllers/camera_zoom_service --ros-args'].

1638279769.9756637 [ERROR] [joint_state_controller-4]: process has died [pid 30241, exit code 1, cmd '/home/pierre/reachy_ws/install/reachy_controllers/lib/reachy_controllers/joint_state_controller --ros-args'].

1638279809.0592141 [ERROR] [camera_server-9]: process has died [pid 30261, exit code -2, cmd '/home/pierre/reachy_ws/install/reachy_sdk_server/lib/reachy_sdk_server/camera_server --ros-args'].

1638279809.0595241 [ERROR] [arm_kinematics_service-3]: process has died [pid 30239, exit code -2

In [21]:
for line in lines:
    if 'reachy_pyluos_hal.reachy.MissingContainerError' in line:
        print(line)

1638279769.9094121 [joint_state_controller-4] reachy_pyluos_hal.reachy.MissingContainerError: Could not find given devices [<DynamixelMotor type="MX106" id=20 limits=(-1.05, 3.14)>, <DynamixelMotor type="MX106" id=21 limits=(-1.57, 1.75)>, <DynamixelMotor type="MX64" id=22 limits=(-1.57, 1.57)>, <DynamixelMotor type="MX106" id=23 limits=(0.0, 2.18)>, <DynamixelMotor type="MX28" id=24 limits=(-1.75, 1.75)>, <DynamixelMotor type="MX28" id=25 limits=(-0.79, 0.79)>, <DynamixelMotor type="AX18" id=26 limits=(-0.61, 0.95)>, <DynamixelMotor type="AX18" id=27 limits=(-1.7, 0.9)>, <ForceSensor id=20>]!



In [27]:
err = str([l for l in lines if 'reachy_pyluos_hal.reachy.MissingContainerError' in l])

In [37]:
test = err.split('devices')[1].split('>, <')

In [63]:
print(test)

[' [<DynamixelMotor type="MX106" id=20 limits=(-1.05, 3.14)', 'DynamixelMotor type="MX106" id=21 limits=(-1.57, 1.75)', 'DynamixelMotor type="MX64" id=22 limits=(-1.57, 1.57)', 'DynamixelMotor type="MX106" id=23 limits=(0.0, 2.18)', 'DynamixelMotor type="MX28" id=24 limits=(-1.75, 1.75)', 'DynamixelMotor type="MX28" id=25 limits=(-0.79, 0.79)', 'DynamixelMotor type="AX18" id=26 limits=(-0.61, 0.95)', 'DynamixelMotor type="AX18" id=27 limits=(-1.7, 0.9)', "ForceSensor id=20>]!\\n']"]


In [93]:
def identify_missing_pieces(error_msg: str):
    return [{'DxlMotor': get_id_from_piece(piece)} for piece in error_msg if 'DynamixelMotor' in piece] + [{'ForceSensor': get_id_from_piece(piece)} for piece in error_msg if 'ForceSensor' in piece]

In [51]:
def get_id_from_piece(piece: str):
    return int(piece.split('id=')[1][:2])

In [64]:
for piece in test:
    print(identify_missing_piece(piece))

{'DxlMotor': 20}
{'DxlMotor': 21}
{'DxlMotor': 22}
{'DxlMotor': 23}
{'DxlMotor': 24}
{'DxlMotor': 25}
{'DxlMotor': 26}
{'DxlMotor': 27}
{'ForceSensor': 20}


In [56]:
def is_missing_container(log_msg: str):
    for message in log_msg:
        if 'reachy_pyluos_hal.reachy.MissingContainerError' in message:
            return True
    return False

In [67]:
def get_missing_container_msg(log_msg: str):
    err_msg = str([msg for msg in log_msg if 'reachy_pyluos_hal.reachy.MissingContainerError' in msg])
    return err_msg.split('devices')[1].split('>, <')

In [92]:
identify_missing_pieces(get_missing_container_msg(lines))

[{'DxlMotor': 20},
 {'DxlMotor': 21},
 {'DxlMotor': 22},
 {'DxlMotor': 23},
 {'DxlMotor': 24},
 {'DxlMotor': 25},
 {'DxlMotor': 26},
 {'DxlMotor': 27},
 {'ForceSensor': 20}]

In [71]:
get_missing_container_msg(lines) == test

True

In [77]:
error_msg = get_missing_container_msg(lines)

In [79]:
for piece in error_msg:
    if 'DynamixelMotor' in piece:
        print(piece)

 [<DynamixelMotor type="MX106" id=20 limits=(-1.05, 3.14)
DynamixelMotor type="MX106" id=21 limits=(-1.57, 1.75)
DynamixelMotor type="MX64" id=22 limits=(-1.57, 1.57)
DynamixelMotor type="MX106" id=23 limits=(0.0, 2.18)
DynamixelMotor type="MX28" id=24 limits=(-1.75, 1.75)
DynamixelMotor type="MX28" id=25 limits=(-0.79, 0.79)
DynamixelMotor type="AX18" id=26 limits=(-0.61, 0.95)
DynamixelMotor type="AX18" id=27 limits=(-1.7, 0.9)


In [80]:
{'DxlMotor': get_id_from_piece(piece) for piece in error_msg if 'DynamixelMotor' in piece}

{'DxlMotor': 27}

In [84]:
identify_missing_pieces(test)

In [20]:
from os import path
import os

In [21]:
path.abspath(os.getcwd())

'/home/pierre/dev/RAP-2021/dashboard'

In [4]:
os.getcwd()

'/home/pierre/dev/RAP-2021/dashboard'

In [5]:
path.abspath('~/dev')

'/home/pierre/dev/RAP-2021/dashboard/~/dev'

In [22]:
home_path = path.expanduser('~')

In [23]:
ros_log_path = home_path + '/.ros/log'

In [24]:
ros_log_path

'/home/pierre/.ros/log'

In [25]:
walk = [file[0] for file in os.walk(ros_log_path)]

In [27]:
# walk

In [29]:
log_folders = walk[1:]
log_folders

['/home/pierre/.ros/log/2021-11-30-14-40-59-661651-reachy-xprize-29832',
 '/home/pierre/.ros/log/2021-12-01-08-11-39-812015-reachy-xprize-1803',
 '/home/pierre/.ros/log/2021-11-30-17-33-19-329442-reachy-xprize-33040',
 '/home/pierre/.ros/log/2021-11-30-17-39-05-388060-reachy-xprize-34963',
 '/home/pierre/.ros/log/2021-12-02-09-48-19-544804-reachy-xprize-1904',
 '/home/pierre/.ros/log/2021-12-01-07-55-13-096647-reachy-xprize-1840',
 '/home/pierre/.ros/log/2021-12-01-08-13-10-818967-reachy-xprize-2506',
 '/home/pierre/.ros/log/2021-12-01-08-31-53-924545-reachy-xprize-5693',
 '/home/pierre/.ros/log/2021-11-30-17-34-47-090383-reachy-xprize-33474',
 '/home/pierre/.ros/log/2021-11-30-18-05-12-080974-reachy-xprize-35702',
 '/home/pierre/.ros/log/2021-11-30-17-37-53-795651-reachy-xprize-34406',
 '/home/pierre/.ros/log/2021-11-30-10-33-58-737989-reachy-xprize-21188',
 '/home/pierre/.ros/log/2021-11-30-18-01-15-504711-reachy-xprize-35525',
 '/home/pierre/.ros/log/2021-12-01-08-21-11-141206-reach

In [37]:
dates = [p.split('log/')[1].split('-reachy')[0] for p in log_folders]

In [43]:
dtimes = [datetime(int(d.split('-')[0]), int(d.split('-')[1]), int(d.split('-')[2]), int(d.split('-')[3]), int(d.split('-')[4]), int(d.split('-')[5])) for d in dates]

In [44]:
dtimes 

[datetime.datetime(2021, 11, 30, 14, 40, 59),
 datetime.datetime(2021, 12, 1, 8, 11, 39),
 datetime.datetime(2021, 11, 30, 17, 33, 19),
 datetime.datetime(2021, 11, 30, 17, 39, 5),
 datetime.datetime(2021, 12, 2, 9, 48, 19),
 datetime.datetime(2021, 12, 1, 7, 55, 13),
 datetime.datetime(2021, 12, 1, 8, 13, 10),
 datetime.datetime(2021, 12, 1, 8, 31, 53),
 datetime.datetime(2021, 11, 30, 17, 34, 47),
 datetime.datetime(2021, 11, 30, 18, 5, 12),
 datetime.datetime(2021, 11, 30, 17, 37, 53),
 datetime.datetime(2021, 11, 30, 10, 33, 58),
 datetime.datetime(2021, 11, 30, 18, 1, 15),
 datetime.datetime(2021, 12, 1, 8, 21, 11),
 datetime.datetime(2021, 11, 30, 10, 42, 51),
 datetime.datetime(2021, 12, 1, 8, 21, 43),
 datetime.datetime(2021, 11, 30, 14, 55, 22),
 datetime.datetime(2021, 11, 30, 17, 32, 29),
 datetime.datetime(2021, 11, 30, 17, 34, 17),
 datetime.datetime(2021, 11, 30, 17, 45, 29),
 datetime.datetime(2021, 12, 1, 8, 13, 36),
 datetime.datetime(2021, 12, 1, 8, 35),
 datetime.dat

In [16]:
from datetime import datetime
from datetime import date

In [45]:
dtimes.sort()

In [46]:
dtimes

[datetime.datetime(2021, 11, 30, 10, 33, 58),
 datetime.datetime(2021, 11, 30, 10, 35, 19),
 datetime.datetime(2021, 11, 30, 10, 42, 51),
 datetime.datetime(2021, 11, 30, 14, 40, 59),
 datetime.datetime(2021, 11, 30, 14, 41, 51),
 datetime.datetime(2021, 11, 30, 14, 42, 31),
 datetime.datetime(2021, 11, 30, 14, 55, 22),
 datetime.datetime(2021, 11, 30, 17, 32, 29),
 datetime.datetime(2021, 11, 30, 17, 33, 19),
 datetime.datetime(2021, 11, 30, 17, 34, 17),
 datetime.datetime(2021, 11, 30, 17, 34, 47),
 datetime.datetime(2021, 11, 30, 17, 37, 53),
 datetime.datetime(2021, 11, 30, 17, 39, 5),
 datetime.datetime(2021, 11, 30, 17, 44, 35),
 datetime.datetime(2021, 11, 30, 17, 45, 29),
 datetime.datetime(2021, 11, 30, 18, 1, 15),
 datetime.datetime(2021, 11, 30, 18, 5, 12),
 datetime.datetime(2021, 12, 1, 7, 55, 13),
 datetime.datetime(2021, 12, 1, 8, 11, 39),
 datetime.datetime(2021, 12, 1, 8, 13, 10),
 datetime.datetime(2021, 12, 1, 8, 13, 36),
 datetime.datetime(2021, 12, 1, 8, 20, 25),
 